In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w3p1/train.csv
/kaggle/input/2022-ai-w3p1/test.csv
/kaggle/input/2022-ai-w3p1/submit_sample.csv


In [2]:
import torch
import torch.optim as optim

In [3]:
import random
import torch

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [4]:
train = pd.read_csv('../input/2022-ai-w3p1/train.csv')
test = pd.read_csv('../input/2022-ai-w3p1/test.csv')
submit = pd.read_csv('../input/2022-ai-w3p1/submit_sample.csv')

In [5]:
train

,year,avgTemp,minTemp,maxTemp,rainFall,avgPrice
0,20100101,-4.9,-11.0,0.9,0.0,2123
1,20100102,-3.1,-5.5,5.5,0.8,2123
2,20100103,-2.9,-6.9,1.4,0.0,2123
3,20100104,-1.8,-5.1,2.2,5.9,2020
4,20100105,-5.2,-8.7,-1.8,0.7,2060
...,...,...,...,...,...,...
2552,20161227,1.2,-3.2,4.5,2.1,4226
2553,20161228,-1.4,-6.5,4.8,0.0,4200
2554,20161229,-1.3,-4.5,2.1,0.2,4224
2555,20161230,-1.4,-7.0,4.6,0.0,4219


In [6]:
trainX = train.drop(['avgPrice', 'year'], axis=1)
trainY = train['avgPrice']
testX = test.drop('year', axis=1)

In [7]:
x_train = torch.FloatTensor(trainX.to_numpy())
y_train = torch.FloatTensor(trainY.to_numpy().reshape(-1,1))
x_test = torch.FloatTensor(testX.to_numpy())

In [8]:
# 학습을 통해 W와 b 값을 찾겠다. 일단 0으로 초기화
W = torch.zeros([x_train.shape[1],1], requires_grad=True)
print(W)

b = torch.zeros(1, requires_grad=True)
print(b)

tensor([[0.],
        [0.],
        [0.],
        [0.]], requires_grad=True)
tensor([0.], requires_grad=True)


In [9]:
hypothesis = torch.matmul(x_train, W) + b
print(hypothesis)

tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)


In [10]:
print((hypothesis - y_train) ** 2)

tensor([[ 4507129.],
        [ 4507129.],
        [ 4507129.],
        ...,
        [17842176.],
        [17799960.],
        [17799960.]], grad_fn=<PowBackward0>)


In [11]:
cost = torch.mean((hypothesis - y_train) ** 2)
print(cost)

tensor(12013341., grad_fn=<MeanBackward0>)


In [12]:
optimizer = optim.SGD([W, b], lr=0.001)

In [13]:
nb_epochs = 100000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = torch.matmul(x_train, W) + b
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
'''
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W:{}'.format(
            epoch, nb_epochs, torch.mean(cost)
        ))
 '''

"\n    if epoch % 100 == 0:\n        print('Epoch {:4d}/{} W:{}'.format(\n            epoch, nb_epochs, torch.mean(cost)\n        ))\n "

In [14]:
x_test

tensor([[ 4.0000, -1.4000,  9.5000,  0.1000],
        [-1.7000, -5.4000,  1.4000,  3.3000],
        [ 1.4000, -2.0000,  5.0000,  0.0000],
        [ 3.6000, -2.5000, 11.1000,  0.1000],
        [ 5.0000, -1.6000, 11.3000,  0.0000],
        [ 7.3000,  0.7000, 13.6000,  0.7000],
        [12.3000,  7.2000, 17.7000,  0.0000],
        [19.8000, 11.9000, 27.7000,  0.0000],
        [17.5000, 12.3000, 24.5000,  2.9000],
        [20.6000, 11.8000, 28.3000,  0.0000],
        [18.4000, 15.9000, 21.2000,  5.5000],
        [24.7000, 18.6000, 31.5000,  0.0000],
        [26.1000, 24.1000, 28.5000, 26.7000],
        [26.0000, 22.2000, 30.2000, 11.3000],
        [25.6000, 23.5000, 28.4000, 11.6000],
        [23.2000, 17.9000, 29.6000,  0.0000],
        [21.3000, 16.0000, 27.6000,  0.0000],
        [17.3000,  9.9000, 25.9000,  0.0000],
        [20.4000, 16.3000, 26.5000,  0.0000],
        [12.7000,  6.9000, 19.5000,  0.0000],
        [ 5.5000,  1.2000, 10.3000,  0.0000],
        [ 5.3000,  0.1000, 10.7000

In [15]:
submit['Expected'] = (torch.matmul(x_test, W) + b).detach()

In [16]:
submit.to_csv('submit.csv', index=False)